### 参数初始化

In [34]:
import numpy as np
import random


NP = 50  # 个体数目
D = 10  # 目标函数中变量的个数
G = 200  # 最大的进化迭代数
Variation = 0.4  # 变异算子
CR = 0.1  # 交叉算子
Threshold = 1e-6  # 阈值
F=0.5

Left = -20
Right = 20

x = np.random.rand(10, 50) * (Right - Left) + Left  #这个操作是在[Left,Right]之间产生随机数

# 适应度函数
def function(x):
    return sum(x ** 2)
# 求适应度
def fitness(x):
    result=np.empty(x.shape[1]) # 记录种群中个体的适应度
    # 计算每个个体的适应度
    for i in range(0,x.shape[1]):
        result[i]=function(x[:,i])
    return result
# 变异
def variation(x,F):

    # 初始化变异个体
    v = np.zeros(x.shape)
    for i in range(0,x.shape[1]):

        # 表示的是在0-NP范围内，随机生成三个整数，作为索引，且保证三个索引不与当前的循环次数i重复（通过这种方式来确保至少有一个个体是变异了的）
        randoms=random.sample(range(0,x.shape[1]),3)
        while randoms[0]==i or randoms[1]==i or randoms[2]==i :
            randoms=random.sample(range(0,x.shape[1]),3)
        # 计算变异的个体
        v[:,i]=x[:,randoms[0]]+(x[:,randoms[1]]-x[:,randoms[2]])*F
    return v

# 交叉
def cross(x,v,CR):
    u = np.zeros((D, NP))
    rate=np.random.rand()
    for i in range(0,x.shape[0]):
        if rate<=CR or i==rate:
           u[i,:]=v[i,:]
        else:
           u[i,:]=x[i,:]
    return u
# 边界处理
def boundary_process(x,left,right):
    for i in range(0,x.shape[0]):
        for j in range(0,x.shape[1]):
            if x[i][j] < left or x[i][j]>right:
                x[i][j]= random.random()
    return x

### suanfa

In [48]:
x_fitness=fitness(x)

for i in range(0,10000):
    v=variation(x,F)
    #print("变异后的形状",v.shape)
    c=cross(x,v,CR)
    #print("交叉后的形状",v.shape)
    c=boundary_process(c,Left,Right)
    c_fitness=fitness(c)
    # 若超出边界则重新生成
    for m in range(0,x.shape[1]):
        if c_fitness[m]<x_fitness[m]:
            x[:,m]=c[:,m]
            x_fitness[m]=c_fitness[m]
    index=np.argmin(x_fitness)
    print(f"第{i+1}次演化时的",f"最小值的索引为：{index}")
    print(f"最小值为：{x_fitness[index]}")
    print("个体为：",x[:,index])

第1次演化时的 最小值的索引为：3
最小值为：0.0009029738907998152
个体为： [-0.00574081  0.01061426  0.00585804 -0.01243098  0.0113801  -0.0085333
  0.0144603  -0.00404379  0.01186027  0.00025669]
第2次演化时的 最小值的索引为：3
最小值为：0.0009029738907998152
个体为： [-0.00574081  0.01061426  0.00585804 -0.01243098  0.0113801  -0.0085333
  0.0144603  -0.00404379  0.01186027  0.00025669]
第3次演化时的 最小值的索引为：3
最小值为：0.0009029738907998152
个体为： [-0.00574081  0.01061426  0.00585804 -0.01243098  0.0113801  -0.0085333
  0.0144603  -0.00404379  0.01186027  0.00025669]
第4次演化时的 最小值的索引为：3
最小值为：0.0009029738907998152
个体为： [-0.00574081  0.01061426  0.00585804 -0.01243098  0.0113801  -0.0085333
  0.0144603  -0.00404379  0.01186027  0.00025669]
第5次演化时的 最小值的索引为：3
最小值为：0.0009029738907998152
个体为： [-0.00574081  0.01061426  0.00585804 -0.01243098  0.0113801  -0.0085333
  0.0144603  -0.00404379  0.01186027  0.00025669]
第6次演化时的 最小值的索引为：17
最小值为：0.0009029738907998149
个体为： [-0.00574081  0.01061426  0.00585804 -0.01243098  0.0113801  -0.0085333
  0.0144603  -0.0

In [ ]:
import random
for i in range(0,G):
    randoms=random.sample(range(0,NP),3)# 表示的是在0-NP范围内，随机生成三个整数，作为索引



### 算法

In [32]:
print(np.random.rand(1))
print(random.random())

[0.46250776]
0.42495052135999645
